<a href="https://colab.research.google.com/github/sbogde/pandamonium/blob/main/dmi_04_pollution_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import time
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

import folium
from folium.plugins import MarkerCluster

from geopy.geocoders import Nominatim

warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
if 'google.colab' in str(get_ipython()):
    from google.colab import drive, data_table
    drive.mount('/content/drive', force_remount=True)
    file_path = '/content/drive/My Drive/Colab Notebooks/data-mining/data/global_air_pollution.csv'
    data_table.enable_dataframe_formatter()
else:
    file_path = './data/global_air_pollution.csv'  # Local path

Mounted at /content/drive


In [4]:
df = pd.read_csv(file_path)
print("Dataset shape:", df.shape)

Dataset shape: (23463, 12)


In [5]:
df.head()

,Country,City,AQI Value,AQI Category,CO AQI Value,CO AQI Category,Ozone AQI Value,Ozone AQI Category,NO2 AQI Value,NO2 AQI Category,PM2.5 AQI Value,PM2.5 AQI Category
0,Russian Federation,Praskoveya,51,Moderate,1,Good,36,Good,0,Good,51,Moderate
1,Brazil,Presidente Dutra,41,Good,1,Good,5,Good,1,Good,41,Good
2,Italy,Priolo Gargallo,66,Moderate,1,Good,39,Good,2,Good,66,Moderate
3,Poland,Przasnysz,34,Good,1,Good,34,Good,0,Good,20,Good
4,France,Punaauia,22,Good,0,Good,22,Good,0,Good,6,Good


In [7]:
pollutant_cols = ['CO AQI Value', 'Ozone AQI Value', 'NO2 AQI Value', 'PM2.5 AQI Value']

df[pollutant_cols] = df[pollutant_cols].fillna(0)

df[pollutant_cols].describe()

,CO AQI Value,Ozone AQI Value,NO2 AQI Value,PM2.5 AQI Value
count,23463.000000,23463.000000,23463.000000,23463.000000
mean,1.368367,35.193709,3.063334,68.519755
std,1.832064,28.098723,5.254108,54.796443
min,0.000000,0.000000,0.000000,0.000000
25%,1.000000,21.000000,0.000000,35.000000
50%,1.000000,31.000000,1.000000,54.000000
75%,1.000000,40.000000,4.000000,79.000000
max,133.000000,235.000000,91.000000,500.000000


In [ ]:
geolocator = Nominatim(user_agent="air_pollution_clustering")

def get_lat_lon(city, country):
    try:
        location = geolocator.geocode(f"{city}, {country}", timeout=10)
        if location:
            return location.latitude, location.longitude
        else:
            return np.nan, np.nan
    except Exception as e:
        return np.nan, np.nan

if 'Latitude' not in df.columns or 'Longitude' not in df.columns:
    latitudes = []
    longitudes = []
    for idx, row in df.iterrows():
        lat, lon = get_lat_lon(row['City'], row['Country'])
        latitudes.append(lat)
        longitudes.append(lon)
        time.sleep(1)
    df['Latitude'] = latitudes
    df['Longitude'] = longitudes

df = df.dropna(subset=['Latitude', 'Longitude'])
print("After geocoding, dataset shape:", df.shape)